# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../output_data/city_weather.csv")


city_weather_df=city_weather_df[["City","Lat", "Lng","Max Temp", "Humidity","Cloudiness","Wind Speed", "Country","Date"]]

city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Gambier,-37.8333,140.7667,67.91,52,100,18.41,AU,1645329489
1,Acapulco de Juárez,16.8634,-99.8901,77.99,68,3,6.04,MX,1645329489
2,Lander,42.8330,-108.7307,35.98,52,0,6.51,US,1645329489
3,Torbay,47.6666,-52.7314,26.98,78,100,16.11,CA,1645329490
4,Cape Town,-33.9258,18.4232,63.27,81,20,4.61,ZA,1645329490
...,...,...,...,...,...,...,...,...,...
558,Isla Mujeres,21.2311,-86.7310,75.16,76,0,14.29,MX,1645329615
559,Leshukonskoye,64.8989,45.7622,25.70,96,100,5.99,RU,1645329734
560,Kamina,-8.7386,24.9906,62.20,98,100,3.24,CD,1645329734
561,Mineiros,-17.5694,-52.5511,64.92,96,46,5.41,BR,1645329735


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#set location
locations = city_weather_df[["Lat", "Lng"]].astype(float)
# Set the humidity
humidity = city_weather_df["Humidity"].astype(float)

In [4]:
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
#Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
## Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

city_weather_df=city_weather_df.loc[((city_weather_df["Max Temp"]<80) & (city_weather_df["Max Temp"]>70))]
city_weather_df=city_weather_df.loc[city_weather_df["Wind Speed"]<10]
city_weather_df=city_weather_df.loc[city_weather_df["Cloudiness"]==0]
city_weather_df=city_weather_df.dropna()
city_weather_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Bhag,29.0415,67.8239,71.28,21,0,4.79,PK,1645329499
231,Barra de Tecoanapa,16.5167,-98.7500,78.94,72,0,3.78,MX,1645329581
417,Turbat,26.0023,63.0440,74.08,30,0,3.06,PK,1645329677


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=pd.DataFrame(city_weather_df)
hotel_df["Hotel Name"]=""
hotel_df=hotel_df.dropna()

In [12]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" with lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()

   # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…